In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [28]:
df = pd.read_csv('telecom_data.csv')
df

,Unnamed: 0,index,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,1,5575-GNVDE,Male,No,No,No,34,Yes,No,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
1,1,2,3668-QPYBK,Male,No,No,No,2,Yes,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
2,2,3,7795-CFOCW,Male,No,No,No,45,No,No phone service,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
3,3,4,9237-HQITU,Female,No,No,No,2,Yes,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
4,4,5,9305-CDSKC,Female,No,No,No,8,Yes,Yes,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.50,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6038,6038,7037,2569-WGERO,Female,No,No,No,72,Yes,No,...,No,No,No,No,Two year,Yes,Bank transfer (automatic),21.15,1419.40,No
6039,6039,7038,6840-RESVB,Male,No,Yes,Yes,24,Yes,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,No
6040,6040,7039,2234-XADUH,Female,No,Yes,Yes,72,Yes,Yes,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,No
6041,6041,7041,8361-LTMKD,Male,Yes,Yes,No,4,Yes,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,Yes


In [3]:
# Changed 'No internet service' to 'No'
cols = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
for col in cols:
    for i in df.index:
        if df.loc[i,col] == 'No internet service':
            df.loc[i] = 'No'

In [4]:
# Created 'tenure_group'
def ten_group(x):
    t = range(0,61,12)
    if x >= t[5]:
        return '60+'
    elif x >= t[4]:
        return '48-60'
    elif x >= t[3]:
        return '36-48'
    elif x >= t[2]:
        return '24-36'
    elif x >= t[1]:
        return '12-24'
    else:
        return '0-12'

df['tenure_group'] = df['tenure'].apply(ten_group)
df.groupby('tenure_group').agg({'index':'count'})

,index
tenure_group,
0-12,1573
12-24,844
24-36,719
36-48,670
48-60,775
60+,1462


In [5]:
# Preprocess categorical columns
col_to = []
for col in df.columns:
    if df[col].nunique() == 2:
        col_to.append(col)
            
for col in col_to[1:]:
    df[col] = df[col].apply(lambda x: 1 if x == 'Yes' else 0)

df['gender'] = df['gender'].apply(lambda x: 1 if x == 'Female' else 0)

print('Количество бинарных переменных', len(col_to))

Количество бинарных переменных 13


In [6]:
# Encoding other categorical variables and drop customerID
enc_cols = []

for col in df.columns:
    if 2 < df[col].nunique() < 10:
        enc_cols.append(col)

print('Количество переменных для one hot  = ', len(enc_cols))

for col in enc_cols:
    one_hot = pd.get_dummies(df[col], sparse=True, prefix = col)
    df = df.join(one_hot)

df = df.drop(columns = enc_cols)
df = df.drop(columns = df.columns[0:2].tolist())
df = df.drop(columns = 'customerID')

Количество переменных для one hot  =  5


In [7]:
# Split our data to 5 folds
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

In [8]:
# Grid search for log regression
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

col = 'Churn'
y = df[col]
x = df.drop(columns = col)

cc = []
c = 0
while c <= 1:
    c += 0.01
    cc.append(round(c,2))

params = {'C':cc}

score = 'roc_auc'
model = LogisticRegression(max_iter = 1000)

cv = GridSearchCV(model, params, score, cv=kf)
cv.fit(x, y)
cv.best_params_, cv.best_score_

({'C': 0.17}, 0.93175448180099)

In [9]:
# Grid search for forrest
from sklearn.ensemble import RandomForestClassifier

params = {'max_depth':list(range(3,22,3)),
         'n_estimators':list(range(10,160,20))}

score = 'roc_auc'
model = RandomForestClassifier()

cv = GridSearchCV(model, params, score, cv=kf)
cv.fit(x, y)
cv.best_params_, cv.best_score_

({'max_depth': 9, 'n_estimators': 130}, 0.9311415087967587)

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
    
col = 'Churn'
y = df[col]
x = df.drop(columns = col)

lr_model = LogisticRegression(max_iter = 1000, C = 0.17)

print(cross_val_score(lr_model, x, y, cv=kf, scoring=('roc_auc')).mean())

0.93175448180099


In [13]:
# log regressin have higher accuracy, so we'll use it to make our predictions
df['Churn_predicted'] = cross_val_predict(lr_model, x, y, cv=kf)

In [14]:
# Confusion matrix for our results
from sklearn.metrics import confusion_matrix
from matplotlib.pyplot import imshow

cm = pd.DataFrame(confusion_matrix(df['Churn'], df['Churn_predicted']))
print('Confusion matrix for churn')
print('    predicted')
print(cm)

Confusion matrix for churn
    predicted
      0     1
0  3958   216
1   451  1418


In [15]:
# find mean payment
p = df['MonthlyCharges'].mean()
print('customers pay "p"  =', p)

customers pay "p"  = 62.85776104583815


In [16]:
# Function to find total profit
def profit(p, discount, acceptance):
    gains = cm.loc[1,1] * p * (acceptance / 100)
    costs = (cm.loc[1,1] + cm.loc[0,1]) * (discount / 100) * p
    losses = (cm.loc[1,0] * p) + (cm.loc[1,1] * (1 - (acceptance / 100)) * (1 - (discount / 100)) * p)
    return (f'Total profit for discount {discount} and acceptance {acceptance} = ', (gains - costs) - losses)

In [17]:
# Total profit for our 2 strategies
print(profit(p = p, discount = 20, acceptance = 75))
print(profit(p = p, discount = 30, acceptance = 90))

('Total profit for discount 20 and acceptance 75 = ', 132.00129819625727)
('Total profit for discount 30 and acceptance 90 = ', 14818.08858894588)


In [18]:
# Description strategy B
print(f'a) TP = {cm.loc[1,1]}, FP = {cm.loc[0,1]}, TN = {cm.loc[0,0]}, FN = {cm.loc[1,0]}' )
print(f'b) Losses for doing nothing  = {(cm.loc[1,1] + cm.loc[1,0]) * p}')
print(f'c) Total gains = {cm.loc[1,1] * p * (90 / 100)}')
print(f'd) Total costs = {(cm.loc[1,1] + cm.loc[0,1]) * (1 - (30 / 100)) * (1 - (30 / 100))* p}')
print(f'e) Total losses = {(cm.loc[1,0] * p) + (cm.loc[1,1] * (1 - (90 / 100)) * (1 - (30 / 100)) * p)}')
print(f'f) {profit(p = p, discount = 30, acceptance = 90)}')
print(f'g) Profit per customer = {14818.08858894588 / (cm.loc[1,1]*0.9 + cm.loc[0,1] + cm.loc[0,0])}')

a) TP = 1418, FP = 216, TN = 3958, FN = 451
b) Losses for doing nothing  = 117481.15539467151
c) Total gains = 80219.07464669865
d) Total costs = 50327.694958960776
e) Total losses = 34588.111593082904
f) ('Total profit for discount 30 and acceptance 90 = ', 14818.08858894588)
g) Profit per customer = 2.7188155643730285


In [19]:
# predict probability to change threshold
df['Churn_probability'] = ''
df['Churn_probability'] = cross_val_predict(estimator = lr_model, X = x, y = y, cv=kf, method='predict_proba')

In [20]:
# create predictions with different thresholds
thresholds = []
cols_thresh = []
i = 0.1
while i <= 0.9:
    thresholds.append(round(i,1))
    i += 0.1
for threshold in thresholds:
    df[f'threshold_{threshold}'] = ''
    df[f'threshold_{threshold}'] = df['Churn_probability'].apply(lambda x: 0 if x>= threshold else 1)
    cols_thresh.append(f'threshold_{threshold}') 

In [21]:
for thr in cols_thresh:
    print(thr)
    print()
    print(pd.DataFrame(confusion_matrix(df['Churn'], df[thr])))
    print()

threshold_0.1

      0    1
0  4174    0
1  1019  850

threshold_0.2

      0     1
0  4174     0
1   775  1094

threshold_0.3

      0     1
0  4164    10
1   646  1223

threshold_0.4

      0     1
0  4096    78
1   530  1339

threshold_0.5

      0     1
0  3958   216
1   451  1418

threshold_0.6

      0     1
0  3753   421
1   368  1501

threshold_0.7

      0     1
0  3515   659
1   297  1572

threshold_0.8

      0     1
0  3140  1034
1   195  1674

threshold_0.9

      0     1
0  2652  1522
1   117  1752



In [22]:
# Find threshhold with better accuracy 
def accuracy(real, pred):
    cm = pd.DataFrame(confusion_matrix(real, pred))
    accuracy = (cm.loc[1,1] + cm.loc[0,0]) / (cm.loc[1,1] + cm.loc[1,0] + cm.loc[0,1] + cm.loc[0,0])
    return accuracy

qqq = []
for thresh in cols_thresh:
    qqq.append([thresh, accuracy(real = df['Churn'], pred = df[thresh])])

ac_tab = pd.DataFrame(qqq)
ac_tab[ac_tab[1] == ac_tab[1].max()]

,0,1
3,threshold_0.4,0.899388


In [23]:
# Find the most profitable threshhold
def profitability(real, pred, p):
    cm = pd.DataFrame(confusion_matrix(real, pred))
    gains = cm.loc[1,1] * p * (90 / 100)
    costs = (cm.loc[1,1] + cm.loc[0,1]) * (30 / 100) * p
    losses = (cm.loc[1,0] * p) + (cm.loc[1,1] * (1 - (90 / 100)) * (1 - (30 / 100)) * p)
    return ((gains - costs) - losses)

qqq = []
for thresh in cols_thresh:
    qqq.append([thresh, profitability(real = df['Churn'], pred = df[thresh], p = p)])

prof_tab = pd.DataFrame(qqq)
prof_tab[prof_tab[1] == prof_tab[1].max()]

,0,1
7,threshold_0.8,24012.921875


In [24]:
# Find the most profitable per customer threshhold
def prof_per_cust(real, pred, p):
    cm = pd.DataFrame(confusion_matrix(real, pred))
    gains = cm.loc[1,1] * p * (90 / 100)
    costs = (cm.loc[1,1] + cm.loc[0,1]) * (30 / 100) * p
    losses = (cm.loc[1,0] * p) + (cm.loc[1,1] * (1 - (90 / 100)) * (1 - (30 / 100)) * p)
    return (((gains - costs) - losses)/(cm.loc[1,1]*0.9 + cm.loc[0,1] + cm.loc[0,0]))

qqq = []
for thresh in cols_thresh:
    qqq.append([thresh, prof_per_cust(real = df['Churn'], pred = df[thresh], p = p)])

prof_per_tab = pd.DataFrame(qqq)

print(prof_per_tab[prof_per_tab[1] == prof_per_tab[1].max()])
print()
print(f'ratio is {(prof_per_tab[prof_per_tab[1] == prof_per_tab[1].max()].loc[7,1] / p)}')

               0         1
7  threshold_0.8  4.227181

ratio is 0.06724993838679014


In [25]:
# Same operation with thresholds with approach A
def profitability_2(real, pred, p):
    cm = pd.DataFrame(confusion_matrix(real, pred))
    gains = cm.loc[1,1] * p * (75 / 100)
    costs = (cm.loc[1,1] + cm.loc[0,1]) * (20 / 100) * p
    losses = (cm.loc[1,0] * p) + (cm.loc[1,1] * (1 - (75 / 100)) * (1 - (20 / 100)) * p)
    return ((gains - costs) - losses)

qqq = []
for thresh in cols_thresh:
    qqq.append([thresh, profitability_2(real = df['Churn'], pred = df[thresh], p = p)])

prof_tab_2 = pd.DataFrame(qqq)
prof_tab_2[prof_tab_2[1] == prof_tab_2[1].max()]

,0,1
8,threshold_0.9,12056.118569


In [26]:
# The most profitable is threshold 0.8 for strategy B.
cm8 = pd.DataFrame(confusion_matrix(df['Churn'], df['threshold_0.8']))
print('Data for threshhold 0.8')
print()
print(f'TP = {cm8.loc[1,1]}, FP = {cm8.loc[0,1]}, TN = {cm8.loc[0,0]}, FN = {cm8.loc[1,0]}' )
print()
print(f'Total profit = {prof_tab[prof_tab[1] == prof_tab[1].max()].loc[7,1]}')

Data for threshhold 0.8

TP = 1674, FP = 1034, TN = 3140, FN = 195

Total profit = 24012.921874731095


In [27]:
# The most profitable per customer is threshold 0.8 for strategy B.
cm8 = pd.DataFrame(confusion_matrix(df['Churn'], df['threshold_0.8']))
print('Data for threshhold 0.8')
print()
print(f'TP = {cm8.loc[1,1]}, FP = {cm8.loc[0,1]}, TN = {cm8.loc[0,0]}, FN = {cm8.loc[1,0]}' )
print()
print(f'Profit per customer = {prof_tab[prof_tab[1] == prof_tab[1].max()].loc[7,1] / (cm8.loc[1,1]*0.9 + cm8.loc[0,1] + cm8.loc[0,0])} ')

Data for threshhold 0.8

TP = 1674, FP = 1034, TN = 3140, FN = 195

Profit per customer = 4.227180557464193 
